In [1]:
import cv2
import sys
sys.path.append("../")
from scripts.yolo_predictions import YOLO_Pred

In [ ]:
yolob = YOLO_Pred('../hackathon-info/colabmodels/best100.onnx','../data.yaml')

In [2]:
# Import deepsort

from deep_sort_realtime.deepsort_tracker import DeepSort
tracker = DeepSort(max_age=5)

In [ ]:
input_video_path = '../hackathon-info/processed_videos/stn_hd_1_first_30_seconds_with_5_fps.mp4'
output_video_path = '../hackathon-info/processed_videos/stn_hd_1_first_30_seconds_with_5_fps_yolo_model_predictions_best100.mp4'

cap = cv2.VideoCapture(input_video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use appropriate codec based on desired output format
out = cv2.VideoWriter(output_video_path, fourcc, 1, (int(cap.get(3)), int(cap.get(4))))


while True:
    ret, frame = cap.read()
    if not ret:
        print('unable to read video')
        break

    pred_image = yolob.predictions(frame)

    # cv2.imshow('YOLO',pred_image)
    out.write(pred_image)
    if cv2.waitKey(1) == 27:
        break

cv2.destroyAllWindows()
cap.release()
out.release()

unable to read video


In [2]:
import cv2
import sys
sys.path.append("../")
from scripts.yolo_predictions import YOLO_Pred
from deep_sort_realtime.deepsort_tracker import DeepSort

# Initialize YOLO model
yolob = YOLO_Pred('../hackathon-info/colabmodels/best100.onnx', '../data.yaml')

# Initialize Deep SORT tracker
tracker = DeepSort(max_age=5, )

input_video_path = '../hackathon-info/processed_videos/stn_hd_1_first_30_seconds_with_5_fps.mp4'
output_video_path = '../hackathon-info/processed_videos/stn_hd_1_first_30_seconds_with_5_fps_yolo_model_predictions_best100.mp4'

cap = cv2.VideoCapture(input_video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use appropriate codec based on desired output format
out = cv2.VideoWriter(output_video_path, fourcc, 1, (int(cap.get(3)), int(cap.get(4))))

# Define counting zones (example coordinates)
# zones = {
#     'A_to_B': ((100, 200), (300, 400)),  # Define as ((x1, y1), (x2, y2))
#     'A_to_C': ((500, 200), (700, 400))
# }

# Initialize counters
vehicle_counts = {'A_to_B': 0, 'A_to_C': 0}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print('Unable to read video')
        break

    # Get YOLO predictions
    detections = yolob.get_detections(frame)

    # Format detections for Deep SORT (tlwh: top-left x, top-left y, width, height, confidence, class)
    formatted_detections = []
    for detection in detections:
        x, y, w, h, conf, class_id = detection
        formatted_detections.append([x, y, w, h, conf, class_id])

    # Update tracker with new detections
    tracked_objects = tracker.update_tracks(formatted_detections, frame=frame)

    # Draw bounding boxes and update counters
    for track in tracked_objects:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        bbox = track.to_tlwh()
        class_id = track.det_class
        x, y, w, h = map(int, bbox)
        color = yolob.generate_colors(class_id)
        label = f'{yolob.labels[class_id]} {track_id}'

        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Detect direction and update counters
        # direction = detect_direction(track, frame)
        # if direction:
        #     turn_counters[direction] += 1

    out.write(frame)

    # Display the frame
    # cv2.imshow('YOLOv5 + Deep SORT', frame)
    # if cv2.waitKey(1) == 27:
    #     break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

# Print the final counts
# print('Vehicle Counts:')
# for zone, count in vehicle_counts.items():
#     print(f'{zone}: {count}')


ModuleNotFoundError: No module named 'torch'